In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('/kaggle/input/telecom-churn/telecom_churn.csv')
data

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.70,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.70,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.00,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.00,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.00,3,166.7,113,41.0,7.42,10.1
...,...,...,...,...,...,...,...,...,...,...,...
3328,0,192,1,1,2.67,2,156.2,77,71.7,10.78,9.9
3329,0,68,1,0,0.34,3,231.1,57,56.4,7.67,9.6
3330,0,28,1,0,0.00,2,180.8,109,56.0,14.44,14.1
3331,0,184,0,0,0.00,2,213.8,105,50.0,7.98,5.0


In [5]:
df = data.copy()

In [6]:
X = df.drop(columns=['Churn'])
y = df['Churn']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

___
# Build a model selection module

In [8]:
# Classification models

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

```python
>> selecion_pipe = GMSModule(mode='binary_classification',
metrics=['f1-score', 'accuracy', 'precision'],
include=[LinearRegression(),
RidgeRegression()...]),
data=[X_train, X_test, y_train, y_test],
verbose=True)

>> selection_pipe.run()
>> selection_pipe.name() # LGBMCLassifier
>> selection_pipe.describe() # Linear Regression - 0.94...
>> selection_pipe.evaluation() # F1-score: 0.94
                               # Accuracy: 0.96...
```

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [29]:
"""    === General Model Selection Module (GMSModule) ===    """

"""Brief desc.: A simple, yet efficient module made
to get crucial info about evaluation of each model passed into the GMSModule object.
User can pass any or all wanted models/pipelines, metrics into the object and use
vast variety of methods that this object provides.

This module automates the process of "blind" model selection models/pipelines
and shrinks your model selection process into 3-4 lines of code!
This module provides verbose and elegant description for model selection"""

"""QUICKSTART:
- Create a GMSModule object with all your wanted project data:

>> GMS = GMSModule(mode="classification", metrics=['accuracy', 'f1-score'],
                   include=[LogisticRegression(), RandomForestClassifier()],
                   data=[X_train, X_test, y_train, y_test])

- Then, use one of the object's methods:

>> GMSPipe.best_model()
(out) (RandomForestClassifier(), 1.694659225890316)

Voila!"""



"""
CREDITS
___
Made by:

- Full name: Zhamkov Nikita D.
- Country, city: Russia, Saint-Petersburg
- Project's GitHub repo: https://github.com/plugg1N/gms-module
- Telegram: https://t.me/@jeberkarawita
- Phone: +79119109210
- Email: nikitazhamkov@gmail.com
___
"""


## Import necessary libraries

# Math
import math
# Scorings for 'classification'
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# Scorings for 'regression'
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
# Beautify
from tabulate import tabulate
from tqdm import tqdm
                            

## Main class
class GMSModule:
    def __init__(self, mode: str, include: list, data: list, metrics: list = ['accuracy']) -> None:
        """
        Initiate variables for work / catch errors created by users while creating an object
        
        Args:
            - mode: A 'string' argument, that tells program which task is user solving (see 'valid_modes' for valid values)
                e.g.: mode='classification'
                
            - include: A 'list' arg. that contains a list of models/pipelines
                e.g.: include=[LinearRegression()]
                
            - data: A 'list' arg. that contains training and validation subsets
                e.g.: data=[X_train, X_test, y_train, y_val]
                
            - metrics: A 'list' arg. that contains strings of names of metrics (see 'valid_*_metrics' for valid values)
                e.g.: metrics=['accuracy', 'f1-score']
                
        Returns:
            Nothing.
        """
        
        # Initiate variables
        self.mode = mode
        self.metrics = metrics
        self.include = include
        self.X_train, self.X_test, self.y_train, self.y_test = data
        
        
        ## Catch possible user errors
        
        # Valid values
        valid_modes = ['classification', 'regression']
        valid_classification_metrics = ['accuracy', 'precision', 'f1-score', 'roc-auc']
        valid_regression_metrics = ['recall', 'mae', 'mape', 'mse', 'rmse', 'r2-score']
        valid_data_types = ['list', 'tuple']
        
        # Catch 'mode' error
        if self.mode not in valid_modes:
            raise ValueError(f"'{self.mode}' mode is not valid. Valid modes are in: {valid_modes}")
            
        # Catch 'metrics_classification' error
        if self.mode == 'classification' and any(metric not in valid_classification_metrics for metric in self.metrics):
            raise ValueError(f"{self.metrics} | One or more classification metrics are not valid. Valid metrics for classification are in: {valid_classification_metrics}")
        
        # Catch 'metrics_regression' error
        if self.mode == 'regression' and any(metric not in valid_regression_metrics for metric in self.metrics):
            raise ValueError(f"{self.metrics} | One or more regression metrics are not valid. Valid metrics for regression are in: {valid_regression_metrics}")
            
            

    def evaluate_models(self) -> (list, list):
        """
        Backbone of this module. Evaluate each model on different subsets using metrics given by the user
        
        Args:
            - self: all self.variables from __init__ 
                
        Returns:
            A 'list' of 'tuple' values. Each tuple contains (*model_name, *dict_of_each_metric_evaluation). For both subsets: train and test
        """
        
        # Prepare to inflate lists
        selected_models = self.include
        results_test = []
        results_train = []
        

        # Loop through each given model and use 'tqdm' to keep track of the evaluation process using progress bar
        for model in tqdm(selected_models, desc="Evaluating Each Model", unit=" evaluation"):
            # Fit each model
            model = model.fit(self.X_train, self.y_train)
            
            # Get predictions for train/test subsets
            y_pred_test = model.predict(self.X_test)
            y_pred_train = model.predict(self.X_train)
            
            # Prepare dicts for inflation
            scores_test = {}
            scores_train = {}

            # Evaluate for classification task
            if self.mode == 'classification':
                # For TEST subset
                if 'accuracy' in self.metrics:
                    scores_test['accuracy'] = accuracy_score(self.y_test, y_pred_test)
                if 'precision' in self.metrics:
                    scores_test['precision'] = precision_score(self.y_test, y_pred_test)
                if 'recall' in self.metrics:
                    scores_test['recall'] = recall_score(self.y_test, y_pred_test)
                if 'f1-score' in self.metrics:
                    scores_test['f1-score'] = f1_score(self.y_test, y_pred_test)
                if 'roc-auc' in self.metrics:
                    scores_test['roc-auc'] = roc_auc_score(self.y_test, y_pred_test)
                results_test.append((model, scores_test))
                
                # For TRAIN subset
                if 'accuracy' in self.metrics:
                    scores_train['accuracy'] = accuracy_score(self.y_train, y_pred_train)
                if 'precision' in self.metrics:
                    scores_train['precision'] = precision_score(self.y_train, y_pred_train)
                if 'recall' in self.metrics:
                    scores_train['recall'] = recall_score(self.y_train, y_pred_train)
                if 'f1-score' in self.metrics:
                    scores_train['f1-score'] = f1_score(self.y_train, y_pred_train)
                if 'roc-auc' in self.metrics:
                    scores_train['roc-auc'] = roc_auc_score(self.y_train, y_pred_train)
                results_train.append((model, scores_train))
                
                
            # Evaluate for regression task
            if self.mode == 'regression':
                # For TEST subset
                if 'mae' in self.metrics:
                    scores_test['MAE (Mean Abs. Error)'] = mean_absolute_error(self.y_test, y_pred_test)
                if 'mape' in self.metrics:
                    scores_test['MAPE (Mean Abs. Percent. Error)'] = mean_absolute_percentage_error(self.y_test, y_pred_test)
                if 'mse' in self.metrics:
                    scores_test['MSE (Mean Squared Error)'] = mean_squared_error(self.y_test, y_pred_test)
                if 'rmse' in self.metrics:
                    scores_test['RMSE (Rooted Mean Squared Error)'] = math.sqrt(mean_squared_error(self.y_test, y_pred_test))
                if 'r2-score' in self.metrics:
                    scores_test['r2-score'] = r2_score(self.y_test, y_pred_test)
                results.append((model, scores_test))
                
                # For TRAIN subset
                if 'mae' in self.metrics:
                    scores_train['MAE (Mean Abs. Error)'] = mean_absolute_error(self.y_test, y_pred_train)
                if 'mape' in self.metrics:
                    scores_train['MAPE (Mean Abs. Percent. Error)'] = mean_absolute_percentage_error(self.y_test, y_pred_train)
                if 'mse' in self.metrics:
                    scores_train['MSE (Mean Squared Error)'] = mean_squared_error(self.y_test, y_pred_train)
                if 'rmse' in self.metrics:
                    scores_train['RMSE (Rooted Mean Squared Error)'] = math.sqrt(mean_squared_error(self.y_test, y_pred_train))
                if 'r2-score' in self.metrics:
                    scores_train['r2-score'] = r2_score(self.y_test, y_pred_train)
                results.append((model, scores_train))

        return results_train, results_test
            
            
            

    def best_model(self, print_info: bool = False) -> (str, dict):
        """
        Get name and scores of the best model accroding to sum of the scores
        
        Args:
            - self: variables from __init__ function
                
            - print_info: A 'bool' variable that tells function to print info or to return it for further usage
                
        Returns:
            - if print_info=True: Nothing. Prints model name and sum of scores
            - else: Best model as object and its sum of scores
        """
        
        result_train, result_test = self.evaluate_models()
        
        # Initialize an empty dictionary, prepare for inflation
        model_scores_dict_train = {}
        model_scores_dict_test = {}

        # Iterate through the results and calculate the sum of scores | TRAIN
        for model, scores in result_train:
            score_sum = sum(scores.values())
            model_scores_dict_train[model] = score_sum
            
        # Iterate through the results and calculate the sum of scores | TEST
        for model, scores in result_test:
            score_sum = sum(scores.values())
            model_scores_dict_test[model] = score_sum

        # Find the key with the maximum value in the dictionary
        best_model = max(model_scores_dict_test, key=model_scores_dict_test.get)

        # Print the key of the model with the highest score
        if print_info:
            print(f"Model with the highest score: {best_model} \nWith sum of scores: {model_scores_dict_test[best_model]}")
        else:
            return best_model, model_scores_dict_test[best_model]
        
        
    def describe(self) -> None:
        """
        Print (log) verbose information about each variable passed into the object and all possible returns.
        Needed for user's self analysis and visual control
        
        Args:
            - self: variables from __init__ function
                
        Returns:
            Nothing. Prints description
        """
        
        result_train, result_test = self.evaluate_models()
        best_model_name, best_score = self.best_model()
        
        print("==== DESCRIPTION ====\n")
        print(f"Evaluation mode: {self.mode}\n")
        print(f"From models selected: {self.include}")
        print(f"Metrics for evaluation: {self.metrics}\n\n")
        
        print("# 1. Evaluation metrics for each model passed into the object for | TRAIN / TEST:\n")
        print(tabulate(result_train, headers=["Models (train)", "Metrics (train)"]), "\n\n")
        print(tabulate(result_test, headers=["Models (test)", "Metrics (test)"]), "\n\n")
        
        print("# 2. Best model name and metrics (based on sum of scores):\n")
        print(f"Best model name: {best_model_name}\nBest score of best model: {best_score}")

In [30]:
GMSPipe = GMSModule(mode="classification", metrics=['accuracy', 'f1-score'], include=[LogisticRegression(), RandomForestClassifier()], data=[X_train, X_test, y_train, y_test])
GMSPipe.describe()

Evaluating Each Model: 100%|██████████| 2/2 [00:00<00:00,  3.03 evaluation/s]

==== DESCRIPTION ====

Evaluation mode: classification

From models selected: [LogisticRegression(), RandomForestClassifier()]
Metrics for evaluation: ['accuracy', 'f1-score']


# 1. Evaluation metrics for each model passed into the object for | TRAIN / TEST:

Models (train)            Metrics (train)
------------------------  -----------------------------------------------------------------
LogisticRegression()      {'accuracy': 0.8499399759903962, 'f1-score': 0.24849699398797598}
RandomForestClassifier()  {'accuracy': 0.9995998399359743, 'f1-score': 0.998639455782313} 


Models (test)             Metrics (test)
------------------------  ----------------------------------------------------------------
LogisticRegression()      {'accuracy': 0.8609112709832134, 'f1-score': 0.2564102564102564}
RandomForestClassifier()  {'accuracy': 0.9376498800959233, 'f1-score': 0.761467889908257} 


# 2. Best model name and metrics (based on sum of scores):

Best model name: RandomForestClassifier()
Be